Imports

In [1]:
#imports
from tqdm import tqdm
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from utils import load_data, prepare_all_samples, get_entity_date_pairs, calculate_metrics
from bert_training import tokenize_function, add_special_tokens, make_training_pairs, balance_classes, gold_lookup
from bert_extractor import get_context_window, mark_entity, mark_date, preprocess_input, bert_extraction

Data Loading

In [2]:
# Load data
df = load_data("data/synthetic.csv")
print(f"Loaded {len(df)} records")
#df

Loaded 101 records


In [3]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[0]

Prepared 101 samples


BERT Base

In [4]:
# Load local model and tokenizer
#model_path = "./bert_model_training/base_model"
model_path = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
#Example note
note_text = (
    "Patient diagnosed with asthma on 2024-08-02. "
    "Diabetes was ruled out on 2024-08-02. "
    "Family history of hypertension, last reviewed in 2022. "
    "Patient may have pneumonia, last seen on 2024-08-02."
)

# Example entity and date spans (positions are for illustration)
entity = {'start': 23, 'end': 29, 'label': 'asthma'}  # "asthma"
date = {'start': 33, 'end': 43, 'parsed': '2024-08-02'}  # "2024-08-02"

note_text, entity, date

('Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. Family history of hypertension, last reviewed in 2022. Patient may have pneumonia, last seen on 2024-08-02.',
 {'start': 23, 'end': 29, 'label': 'asthma'},
 {'start': 33, 'end': 43, 'parsed': '2024-08-02'})

In [6]:
#Test context window
context = get_context_window(note_text, entity['start'], date['start'], window_size=50)
print("Context window:\n", context)

Context window:
 Patient diagnosed with asthma on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [7]:
#Test entity marking
entity_text = note_text[entity['start']:entity['end']]
offset = context.find(entity_text)
entity_rel = {'start': offset, 'end': offset + len(entity_text)}

marked_entity = mark_entity(context, entity_rel)
print("Entity marked:\n", marked_entity)

Entity marked:
 Patient diagnosed with [E]asthma[E] on 2024-08-02. Diabetes was ruled out on 2024-08-02. 


In [8]:
#Test date marking
date_text = note_text[date['start']:date['end']]
offset_date = context.find(date_text)
date_rel = {'start': offset_date, 'end': offset_date + len(date_text)}

marked_date = mark_date(context, date_rel)
print("Date marked:\n", marked_date)

Date marked:
 Patient diagnosed with asthma on [D]2024-08-02[D]. Diabetes was ruled out on 2024-08-02. 


In [9]:
#Do full pre-processing
preprocessed = preprocess_input(note_text, entity, date, window_size=20)
print("Preprocessed input:\n", preprocessed)

Preprocessed input:
 ient diagnosed with [E]asthma[E] on [D]2024-08-02[D]. Diabetes


In [ ]:
# Process samples
predictions = []

#for sample in tqdm(samples, desc="Samples"):
    #pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    #for pair in pairs:
        #text = f"Does '{pair['entity_label']}' relate to '{pair['date']}'? {sample['note_text']}..."
        #pred, conf = bert_extraction(text, model, tokenizer)
        #if pred == 1:
            #predictions.append({'entity_label': pair['entity_label'], 'date': pair['date'], 'confidence': conf})

for sample in tqdm(samples, desc="Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        entity = pair['entity']
        date = pair['date_info']
        pred, conf = bert_extraction(sample['note_text'], entity, date, model, tokenizer, window_size=100)
        if pred == 1:
            predictions.append({'entity_label': entity['label'], 'date': date['parsed'], 'confidence': conf})

In [ ]:
#Look at predictions
predictions

In [ ]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics

BERT Finetuning

In [ ]:
#Add special tokens to tokenizer
add_special_tokens(tokenizer)

# Resize model embeddings to match new tokenizer size
model.resize_token_embeddings(len(tokenizer))

In [ ]:
#Do some layer freezing here?

In [ ]:
#Split samples
train_samples, val_samples = train_test_split(samples, test_size=0.2, random_state=42)
print(f"Train: {len(train_samples)}, Val: {len(val_samples)}")

In [ ]:
#Prepare training pairs
training_pairs = make_training_pairs(train_samples, gold_lookup, window_size=100)
val_pairs = make_training_pairs(val_samples, gold_lookup, window_size=100)

In [ ]:
#Balance classes (only training set)
training_pairs = balance_classes(training_pairs, ratio=1.0)

In [ ]:
#Convert to Datasets
train_dataset = Dataset.from_pandas(training_pairs)
val_dataset = Dataset.from_pandas(val_pairs)

In [ ]:
#Tokenization
tokenized_train = train_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)
tokenized_val = val_dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

In [ ]:
#Look at tokenized dataset
tokenized_train

In [ ]:
#Define training args
training_args = TrainingArguments(
    output_dir="./bert_model_training/bert_finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=1,
    load_best_model_at_end=True,
)

In [ ]:
#Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [ ]:
#Train
trainer.train()

In [ ]:
#Process validation samples
predictions = []

for sample in tqdm(val_samples, desc="Validation Samples"):
    pairs = get_entity_date_pairs(sample['entities_list'], sample['dates'])
    for pair in pairs:
        entity = pair['entity']
        date = pair['date_info']
        pred, conf = bert_extraction(sample['note_text'], entity, date, model, tokenizer, window_size=100)
        if pred == 1:
            predictions.append({'entity_label': entity['label'], 'date': date['parsed'], 'confidence': conf})

In [ ]:
#Look at predictions
predictions

In [ ]:
# Calculate metrics
metrics = calculate_metrics(predictions, df)
metrics